## Loading the json file

In [106]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
secret_key = os.getenv('OPEN_AI_API')
client = OpenAI(api_key=secret_key)

with open('recipes.json', 'r') as json_file:
    documents = json.load(json_file)

operations = []
for doc in documents:
    operations.append(doc)

In [75]:
doc

{'name': 'a masterpiece cheesecake',
 'description': 'cheesecake',
 'steps': 'crust: mix well and pat into greased 9-inch spring form pan place in refrigerator while making filling filling: beat well and pour into pie crust bake in 350 degree f oven for 20 minutes only remove and cool cake may be decorated with any variety of fruits , canned or in season',
 'ingredients': 'graham cracker crumbs,butter,powdered sugar,flour,cream cheese,eggs,sugar',
 'id': 194864,
 'text_vector': [-0.014150355011224747,
  -0.08587893843650818,
  -0.026331650093197823,
  -0.06613396108150482,
  -0.07036852836608887,
  0.015694256871938705,
  -0.052334628999233246,
  0.034557752311229706,
  -0.03266534581780434,
  0.038366060703992844,
  -0.11862796545028687,
  -0.01831705868244171,
  -0.0516941137611866,
  -0.08835714310407639,
  0.00948421936482191,
  0.0017496936488896608,
  0.06482762843370438,
  -0.016803676262497902,
  -0.006692420691251755,
  -0.021900704130530357,
  0.17078110575675964,
  -0.074706

## Set up Elasticsearch connection

In [25]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import pandas as pd

es_client= Elasticsearch(
    hosts=["http://localhost:9200"]
)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "ingredients": {"type": "text"},
            "steps": {"type": "text"},
            "name": {"type": "text"},
            "description": {"type": "text"},
            "tags": {"type": "text"},
            "n_ingredients": {"type": "integer"},
            "n_steps": {"type": "integer"},
            "id": {"type": "keyword"},
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "recipes"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'recipes'})

## Add documents to index

In [36]:
for doc in tqdm(operations, total=len(operations)):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/1974 [00:00<?, ?it/s]

## Create end user query

In [71]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [72]:
search_term = "I have some chiles, potatoes, and tomatoes. I only have 30 minutes and I feel like indian food. What should I make?"
vector_search_term = model.encode(search_term)

In [73]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}

In [74]:
res = es_client.search(index=index_name, knn=query, source=["ingredients", "steps", "name", "description", "tags"])
res["hits"]["hits"]
                       

[{'_index': 'recipes',
  '_id': 'h3JX5JAB-0DH_ZzMoiq3',
  '_score': 0.8098093,
  '_source': {'name': 'tide me over   indian chaat  simple veggie salad',
   'description': 'this version of the famous indian chaat (pronounce with the emphasis on the ',
   'ingredients': 'cucumber,potato,chopped tomato,plain yogurt,chat masala,paprika,salt',
   'steps': 'toss everything well in a serving bowl grab a fork or large spoon and eat immediately , or refrigerate until hunger strikes , and then eat ! feel extremely pleased with yourself that you snacked on this instead of that bag of chips you were eyeing proceed to eat several helpings of dessert after dinner as reward'}},
 {'_index': 'recipes',
  '_id': 'CXJX5JAB-0DH_ZzM8ytg',
  '_score': 0.8098093,
  '_source': {'name': 'tide me over   indian chaat  simple veggie salad',
   'description': 'this version of the famous indian chaat (pronounce with the emphasis on the ',
   'ingredients': 'cucumber,potato,chopped tomato,plain yogurt,chat masala,pa

## Putting it all together

In [107]:
def elastic_search(search_term):
    vector_search_term = model.encode(search_term)
    search_query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 10000,
    }

    res = es_client.search(index=index_name, knn=search_query, source=["ingredients", "steps", "name", "description", "tags"])
    result_docs = []
    for hit in res["hits"]["hits"]:
        result_docs.append(hit['_source'])

    return result_docs

def build_prompt(q, search_results):
    prompt_template = """
You are a recipe creator assistant. Answer the QUERY based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUERY.

QUERY: {query}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Recipe title: {doc['name']}\ndescription: {doc['description']}\ningredients: {doc['ingredients']}\nsteps: {doc['steps']}\n\n"
    
    prompt = prompt_template.format(query=q, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [111]:
def rag(q):
    res = elastic_search(q)
    prompt = build_prompt(search_term, res)
    answer = llm(prompt)
    return answer

In [113]:
q = "I have some chiles, potatoes, and tomatoes. I only have 30 minutes and I feel like asian food. What should I make?"
print(rag(q))

Based on the available ingredients and your time constraint, I suggest making a quick Indian-inspired potato and tomato dish with chiles. Here is a simple recipe idea that fits into your 30-minute timeframe:

### Spicy Potato and Tomato Stir-Fry

**Ingredients:**
- Potatoes
- Tomatoes
- Chiles (adjust quantity based on your preferred level of spiciness)
- Olive oil or any cooking oil
- Seasoning salt (or common Indian spices if you have them like turmeric, cumin, and coriander)
- Garlic powder (optional)
- Chili powder
- Black pepper

**Steps:**

1. **Preparation:** 
   - Peel and dice the potatoes into small cubes for faster cooking.
   - Dice the tomatoes.
   - Slice the chiles.

2. **Cooking:**
   - Heat oil in a large skillet or frying pan over medium heat.
   - Add the diced potatoes to the pan and sauté for about 10 minutes, stirring frequently until they start to soften and get a slight browning.
   - Add the sliced chiles and continue to cook for another 2 minutes.
   - Add the